In [1]:
import random
import numpy as np
import tensorflow as tf

In [2]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Activation
from keras.optimizers import RMSprop 

In [3]:
import urllib.request
import ssl
import tensorflow as tf

# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
filepath = tf.keras.utils.get_file('shakespeare.txt', origin=url, cache_dir='.', cache_subdir='')
urllib.request.urlretrieve(url, filepath)
text = open(filepath, 'rb').read().decode(encoding='utf-8')

In [4]:
text=text.lower()

In [5]:
text=text[300000:800000]

In [6]:
characters=sorted(set(text))

In [7]:
char_to_index=dict((c,i)for i,c in enumerate(characters))
index_to_char=dict((i,c)for i,c in enumerate(characters))

In [8]:
seqlen=40
stepsz=3
sen=[]
nextchar=[]

In [9]:
for j in range(0,len(text)-seqlen,stepsz):
    sen.append(text[j:j+seqlen])
    nextchar.append(text[j+seqlen])

In [10]:
x=np.zeros((len(sen),seqlen,len(characters)),dtype=bool)
y=np.zeros((len(sen),len(characters)),dtype=bool)

In [11]:
for i,sent in enumerate(sen):
    for j,char in enumerate(sent):
        x[i,j,char_to_index[char]]=1;
    y[i,char_to_index[nextchar[i]]]=1

In [12]:
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
model.fit(x, y, batch_size=256, epochs=4)
model.save('poet.model')


Epoch 1/4
651/651 [==============================] - 57s 85ms/step - loss: 2.6489
Epoch 2/4
651/651 [==============================] - 55s 84ms/step - loss: 2.2455
Epoch 3/4
651/651 [==============================] - 55s 85ms/step - loss: 2.0964
Epoch 4/4
651/651 [==============================] - 58s 89ms/step - loss: 1.9883
INFO:tensorflow:Assets written to: poet.model/assets


INFO:tensorflow:Assets written to: poet.model/assets


In [13]:
model=tf.keras.models.load_model('poet.model')

In [14]:
def sample(preds,temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [15]:
def textgen(length, temperature):
    start_index=random.randint(0,len(text)-seqlen-1)
    generated=''
    sentence=text[start_index:start_index+seqlen]
    generated+=sentence
    for o in range(length):
        x=np.zeros((1,seqlen,len(characters)),dtype=bool)
        for t,character in enumerate(sentence):
            x[0,t,char_to_index[character]]=1
        predictions=model.predict(x,verbose=0)[0]
        nextindex=sample(predictions,temperature)
        nextcharac=index_to_char[nextindex]
        generated+=nextcharac
        sentence=sentence[1:]+nextcharac
    return generated

In [16]:
print('0.2')
print(textgen(300,0.2))
print('0.4')
print(textgen(300,0.4))
print('0.6')
print(textgen(300,0.6))
print('0.8')
print(textgen(300,0.8))
print('1')
print(textgen(300,1))

0.2
er back of death,
who gently would dissond the will the shall.

king rewward iv:
whis with the print the rise the wich the wind his death
the will the with the with the with low stour his so the beard.

herring:
wher with the with the sond her with the shall the shall the beath the beand.

king rewird:
i with the will the frath the manger
0.4
ave o' the sea, that you might ever do
no miss of i more stand the prailars,
and thil this fare the batt me hand you the sould the mand
that here bread the with i with the weres.

herr sulling:
the sind with the brout the hath she wher sours, thit the wille,
what make the brother lowe star he wing he wourd with i lave
the rere in that the
0.6
full as dearly as i love myself.

king edward ii:
here for for i paring betured frear stand.

wardick:
not aw and thou lath his ging hit me fave then that,
nor what i wind the hish i so farther and no the shave of that,
and her the and he for chend and that this heard;
so, her and to the comps the leaving